In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linprog
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable, LpMinimize
import time
import random

Løsning med inspiration fra https://frknklcsln.medium.com/a-maximum-flow-problem-solution-with-python-34dc3137a3b0 og beskrivelsen i Sisyfos5 modelguide. 
Jeg forestiller mig, at jeg har 7 nodes og 9 forbindelser. Hver node har en produktion og demand mellem 1 og 10.

In [2]:
demands = [random.randint(0,1), random.randint(0,5), random.randint(0,7), random.randint(0,6), random.randint(0,8), random.randint(0,3), random.randint(0,5)]
capacities = [random.randint(0,10), random.randint(0,4), random.randint(0,2), random.randint(0,6), random.randint(0,6), random.randint(0,4), random.randint(0,2)]
lines = [#a,b,max kapacitet a -> b, max kapacitet b -> a
    [0,1,1, 2],
    [0,2,2, 2],
    [1,3,2,2.5],
    [2,3,4,4],
    [2,4,3,2],
    [3,5,4,3],
    [3,6,3,4],
    [4,5,2,2],
    [5,6,3,3]
]


Dette er et lineært optimeringsproblem, som beskrevet i Sisyfos modelguide. Hvis $x_i$ er produktionen i node i som er et tal mellem 0 og produktionskapaciteten $p_i$, $D_i$ er demand, så handler det om at minimere den samlede EENS i alle nodes, som kan defineres som $EENS = \sum_i(D_i-x_i-\sum_j F_{ji})$ hvor $\sum_j F_{ji}$ er det samlede flow ind og ud af noden - dvs F_{ji} er flow fra den j'te til den i'te node, et tal som er mellem -$C_{ij}$ (kapaciteten fra i til j) og kapaciteten fra j til i, $C_{ji}$. Problemet defineres derfor matematisk som: \
Minimering af \
$\sum_i(D_i-x_i-\sum_j F_{ji})$ \
Under betingelserne \
$0 \leq x_i \leq p_i$ production less than capacity \
$-C_{ij} \leq F_{ji} \leq C_{ji}$ flow within capacity limits \
$x_i + \sum_j F_{ji} \leq D_i$ Production and net flow less than demand \
\
Siden det totale flow summerer 0 (fordi der for hver node, der modtager F, er en node der afsender F, og demand ikke varierer under variation af parametrene, er minimeringsbetingelsen ækvivalent til betingelsen minimering af $-\sum_i x_i$ eller maximering af $\sum_i x_i$. Dette lineære optimeringsproblem kan løses computationelt. \
Jeg følger guiden her: https://realpython.com/linear-programming-python. pulp-biblioteket er ideelt til dette.

In [3]:
start_time = time.time()
#initilaize vectors to store variables
x_vec = []
F_vec = []

for i in demands:
    x_vec.append(0)
for i in lines:
    F_vec.append(0)
    
for i in range(len(demands)):
    x_vec[i] = LpVariable(name="x"+str(i), lowBound=0, upBound = capacities[i])
    
for i in range(len(lines)):
    F_vec[i] = LpVariable(name="F"+str(i), lowBound=-lines[i][3], upBound = lines[i][2])
    
#create model
model = LpProblem(name="maxFlow", sense=LpMaximize)

#create objective for model
obj_func = sum(x_vec)
model += obj_func

#constraints on flow and production were made in their initializations. Make constraints on production and net flows being
#less than demand.
for i in range(len(demands)):
    build = x_vec[i]
    #find lines relevant to this node. If they are in index 0, they represent flow out, if they are in index 1, they
    #represent flow in.
    for j in range(len(lines)):
        if(lines[j][0] == i):
            build -= F_vec[j]
        if(lines[j][1] == i):
            build += F_vec[j]
    model += (build <= demands[i], "constraint_demand" + str(i))
    
status = model.solve()
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.05099916458129883 seconds ---


In [4]:
#print model parameters
model

maxFlow:
MAXIMIZE
1*x0 + 1*x1 + 1*x2 + 1*x3 + 1*x4 + 1*x5 + 1*x6 + 0
SUBJECT TO
constraint_demand0: - F0 - F1 + x0 <= 0

constraint_demand1: F0 - F2 + x1 <= 5

constraint_demand2: F1 - F3 - F4 + x2 <= 3

constraint_demand3: F2 + F3 - F5 - F6 + x3 <= 1

constraint_demand4: F4 - F7 + x4 <= 1

constraint_demand5: F5 + F7 - F8 + x5 <= 1

constraint_demand6: F6 + F8 + x6 <= 0

VARIABLES
-2 <= F0 <= 1 Continuous
-2 <= F1 <= 2 Continuous
-2.5 <= F2 <= 2 Continuous
-4 <= F3 <= 4 Continuous
-2 <= F4 <= 3 Continuous
-3 <= F5 <= 4 Continuous
-4 <= F6 <= 3 Continuous
-2 <= F7 <= 2 Continuous
-3 <= F8 <= 3 Continuous
x0 <= 3 Continuous
x1 = 0 Continuous
x2 = 0 Continuous
x3 <= 4 Continuous
x4 <= 6 Continuous
x5 <= 1 Continuous
x6 = 0 Continuous

In [5]:
#gennemsnitlig beregningstid
total = 0
"""
for i in range(1000):
    start_time = time.time()
    status = model.solve()
    total += time.time() - start_time
"""
print("--- %s seconds ---" % (total/1000))

--- 0.0 seconds ---


In [6]:
for var in model.variables():
    print(f"{var.name}: {var.value()}")

F0: 1.0
F1: 2.0
F2: -2.5
F3: 1.0
F4: -2.0
F5: -1.5
F6: 3.0
F7: -1.5
F8: -3.0
x0: 3.0
x1: 0.0
x2: 0.0
x3: 4.0
x4: 1.5
x5: 1.0
x6: 0.0


In [7]:
#Hvad er EENS?
totalEENS = 0
for i in range(len(demands)):
    print("Node " + str(i) + " har demand " + str(demands[i]) + ".")
    print("Node " + str(i) + " producerer selv " + str(model.variables()[9+i].value()) + ".")
    EENS = demands[i]
    EENS -= model.variables()[9+i].value()
    for j in range(len(lines)):
        if(lines[j][0] == i):
            print("Node " + str(i) + " modtager fra " + "node " + str(lines[j][1]) +" " + str(-model.variables()[j].value()) + ".")
            EENS += model.variables()[j].value()
        if(lines[j][1] == i):
            print("Node " + str(i) + " modtager fra " + "node " + str(lines[j][0])+" " + str(model.variables()[j].value()) + ".")
            EENS -= model.variables()[j].value()
    print("Node " + str(i) + " har EENS " + str(EENS) + ".")
    totalEENS += EENS
print(totalEENS)

Node 0 har demand 0.
Node 0 producerer selv 3.0.
Node 0 modtager fra node 1 -1.0.
Node 0 modtager fra node 2 -2.0.
Node 0 har EENS 0.0.
Node 1 har demand 5.
Node 1 producerer selv 0.0.
Node 1 modtager fra node 0 1.0.
Node 1 modtager fra node 3 2.5.
Node 1 har EENS 1.5.
Node 2 har demand 3.
Node 2 producerer selv 0.0.
Node 2 modtager fra node 0 2.0.
Node 2 modtager fra node 3 -1.0.
Node 2 modtager fra node 4 2.0.
Node 2 har EENS 0.0.
Node 3 har demand 1.
Node 3 producerer selv 4.0.
Node 3 modtager fra node 1 -2.5.
Node 3 modtager fra node 2 1.0.
Node 3 modtager fra node 5 1.5.
Node 3 modtager fra node 6 -3.0.
Node 3 har EENS 0.0.
Node 4 har demand 1.
Node 4 producerer selv 1.5.
Node 4 modtager fra node 2 -2.0.
Node 4 modtager fra node 5 1.5.
Node 4 har EENS 0.0.
Node 5 har demand 1.
Node 5 producerer selv 1.0.
Node 5 modtager fra node 3 -1.5.
Node 5 modtager fra node 4 -1.5.
Node 5 modtager fra node 6 3.0.
Node 5 har EENS 0.0.
Node 6 har demand 0.
Node 6 producerer selv 0.0.
Node 6 modt

Sisyfos modelguide hævder at problemet kan løses hurtigere ved at reformulere det som et maxflow problem. I det tilfælde deler de nodes op i nodes med overskudskapacitet og nodes med underskudskapacitet, så \
S = nodes med overskud \
T = nodes med underskud \
De postulerer at når flowet er maksimalt, så er EENS minimal. Jeg er ikke sikker på at jeg forstår at dette nødvendigvis gælder, men det virker intuitiv.
Formuleringen bliver: \
Maksimering af \
$\sum_{j \in T} \sum_i F_{ij}$ flow ind i nodes med underskud \
Under betingelser \
$\sum_j F_{ij} \leq P_i - D_i \forall i \in S$ nettoflow ud af nodes med overskud er ikke større end deres overskud \
$-C_{ij} \leq F_{ji} \leq C_{ji}$ flow within capacity limits \
$\sum_j F_{ij} \leq D_i - P_i \forall i \in T$ nettoflow ind i nodes med underskud er ikke større end deres underskud \
De slipper derfor at optimere på produktionsparametrene. Det gør måske modellen lidt hurtigere?

In [8]:
#initilaize vectors to store variables
F_vec = []

for i in lines:
    F_vec.append(0)
    
for i in range(len(lines)):
    F_vec[i] = LpVariable(name="F"+str(i), lowBound=-lines[i][3], upBound = lines[i][2])
    
#create model
model2 = LpProblem(name="maxFlow", sense=LpMaximize)

obj_func = 0

#find demand and production in each node. 
for i in range(len(demands)):
    hasSurplus = True
    surplus = capacities[i] - demands[i]
    if(surplus < 0):
        hasSurplus = False
    build = 0
    #find lines relevant to this node. If they are in index 0, they represent flow out, if they are in index 1, they
    #represent flow in.
    for j in range(len(lines)):
        if(lines[j][0] == i):
            build -= F_vec[j]
            if(not hasSurplus):
                obj_func -= F_vec[j]
        if(lines[j][1] == i):
            build += F_vec[j]
            if(not hasSurplus):
                obj_func += F_vec[j]
    if(hasSurplus):
        model2 += (-build <= surplus, "constraint_demand" + str(i))
    else:
        model2 += (build <= -surplus, "constraint_demand" + str(i))
    
#create objective for model
model2 += obj_func
model2

maxFlow:
MAXIMIZE
1*F0 + 1*F1 + -1*F2 + -1*F3 + -1*F4 + 0
SUBJECT TO
constraint_demand0: F0 + F1 <= 3

constraint_demand1: F0 - F2 <= 5

constraint_demand2: F1 - F3 - F4 <= 3

constraint_demand3: - F2 - F3 + F5 + F6 <= 3

constraint_demand4: - F4 + F7 <= 5

constraint_demand5: - F5 - F7 + F8 <= 0

constraint_demand6: - F6 - F8 <= 0

VARIABLES
-2 <= F0 <= 1 Continuous
-2 <= F1 <= 2 Continuous
-2.5 <= F2 <= 2 Continuous
-4 <= F3 <= 4 Continuous
-2 <= F4 <= 3 Continuous
-3 <= F5 <= 4 Continuous
-4 <= F6 <= 3 Continuous
-2 <= F7 <= 2 Continuous
-3 <= F8 <= 3 Continuous

In [9]:
status = model2.solve()

In [10]:
for var in model2.variables():
    print(f"{var.name}: {var.value()}")

F0: 1.0
F1: 2.0
F2: -2.5
F3: 1.0
F4: -2.0
F5: -1.5
F6: 3.0
F7: -1.5
F8: -3.0


In [11]:
#Hvad er EENS?
totalEENS = 0
for i in range(len(demands)):
    print("Node " + str(i) + " har demand " + str(demands[i]) + ".")
    print("Node " + str(i) + " producerer selv " + str(capacities[i]) + ".")
    EENS = demands[i] - capacities[i]
    for j in range(len(lines)):
        if(lines[j][0] == i):
            print("Node " + str(i) + " modtager fra " + "node " + str(lines[j][1]) +" " + str(-model2.variables()[j].value()) + ".")
            EENS += model2.variables()[j].value()
        if(lines[j][1] == i):
            print("Node " + str(i) + " modtager fra " + "node " + str(lines[j][0])+" " + str(model2.variables()[j].value()) + ".")
            EENS -= model2.variables()[j].value()
    if(demands[i] > capacities[i]):
        print("Node " + str(i) + " har EENS " + str(EENS) + ".")
        totalEENS += EENS
    else:
        print("Node " + str(i) + " har EENS " + str(0) + ".")
print(totalEENS)

Node 0 har demand 0.
Node 0 producerer selv 3.
Node 0 modtager fra node 1 -1.0.
Node 0 modtager fra node 2 -2.0.
Node 0 har EENS 0.
Node 1 har demand 5.
Node 1 producerer selv 0.
Node 1 modtager fra node 0 1.0.
Node 1 modtager fra node 3 2.5.
Node 1 har EENS 1.5.
Node 2 har demand 3.
Node 2 producerer selv 0.
Node 2 modtager fra node 0 2.0.
Node 2 modtager fra node 3 -1.0.
Node 2 modtager fra node 4 2.0.
Node 2 har EENS 0.0.
Node 3 har demand 1.
Node 3 producerer selv 4.
Node 3 modtager fra node 1 -2.5.
Node 3 modtager fra node 2 1.0.
Node 3 modtager fra node 5 1.5.
Node 3 modtager fra node 6 -3.0.
Node 3 har EENS 0.
Node 4 har demand 1.
Node 4 producerer selv 6.
Node 4 modtager fra node 2 -2.0.
Node 4 modtager fra node 5 1.5.
Node 4 har EENS 0.
Node 5 har demand 1.
Node 5 producerer selv 1.
Node 5 modtager fra node 3 -1.5.
Node 5 modtager fra node 4 -1.5.
Node 5 modtager fra node 6 3.0.
Node 5 har EENS 0.
Node 6 har demand 0.
Node 6 producerer selv 0.
Node 6 modtager fra node 3 3.0.
N

In [12]:
#gennemsnitlig beregningstid
total = 0
"""
for i in range(1000):
    start_time = time.time()
    status = model2.solve()
    total += time.time() - start_time
print("--- %s seconds ---" % (total/1000))
"""

'\nfor i in range(1000):\n    start_time = time.time()\n    status = model2.solve()\n    total += time.time() - start_time\nprint("--- %s seconds ---" % (total/1000))\n'